In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Dropout, Activation, Conv2D, MaxPool2D, GlobalAveragePooling2D
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.applications.inception_v3 import InceptionV3

from dataset import getDataset
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

import os
import cv2
import time
import random

In [2]:
IMG_SIZE = 224

X, y = getDataset(IMG_SIZE)

print(X.shape, y.shape)

datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)


X_aug, y_aug = [], []

cnt = 0
for _X, _y in datagen.flow(X, y):    
    for i in range(_X.shape[0]):
        cnt += 1
        X_aug.append(_X[i])
        y_aug.append(_y[i])
    
    if cnt >= 1000:
        break

X = np.array(X_aug).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
y = np.array(y_aug).reshape(-1, 1)

X.shape, y.shape

(92, 224, 224, 3) (92, 1)


/home/roshan_/anaconda3/envs/tf-gpu/lib/python3.9/site-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/home/roshan_/anaconda3/envs/tf-gpu/lib/python3.9/site-packages/keras_preprocessing/image/image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


((1012, 224, 224, 3), (1012, 1))

In [3]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np

model = VGG16(weights='imagenet', include_top=False)

X_features = []
for img in X:
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = model.predict(x)
    
    X_features.append(features)

print(len(X_features), X_features[0].shape)

h, w, c = X_features[0].shape[1:]
X = np.array(X_features).reshape(-1, h, w, c)
X.shape, y.shape

1012 (1, 7, 7, 512)


((1012, 7, 7, 512), (1012, 1))

In [4]:
model = Sequential()

model.add(Flatten(input_shape=(X.shape[1:])))

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                1605696   
_________________________________________________________________
activation (Activation)      (None, 64)                0         
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                1040      
_________________________________________________________________
activation_1 (Activation)    (None, 16)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0

In [5]:
history = model.fit(X, y, validation_split=0.3, batch_size=32, epochs=50, shuffle=True)

Epoch 1/50
23/23 [==============================] - 1s 23ms/step - loss: 1.8090 - accuracy: 0.7740 - val_loss: 0.0558 - val_accuracy: 0.9770
Epoch 2/50
23/23 [==============================] - 0s 11ms/step - loss: 0.3067 - accuracy: 0.9393 - val_loss: 0.0472 - val_accuracy: 0.9868
Epoch 3/50
23/23 [==============================] - 0s 11ms/step - loss: 0.3148 - accuracy: 0.9562 - val_loss: 0.0592 - val_accuracy: 0.9868
Epoch 4/50
23/23 [==============================] - 0s 11ms/step - loss: 0.0692 - accuracy: 0.9802 - val_loss: 0.0448 - val_accuracy: 0.9901
Epoch 5/50
23/23 [==============================] - 0s 11ms/step - loss: 0.1284 - accuracy: 0.9845 - val_loss: 0.0666 - val_accuracy: 0.9934
Epoch 6/50
23/23 [==============================] - 0s 11ms/step - loss: 0.0941 - accuracy: 0.9859 - val_loss: 0.0055 - val_accuracy: 0.9967
Epoch 7/50
23/23 [==============================] - 0s 11ms/step - loss: 0.0752 - accuracy: 0.9887 - val_loss: 0.0083 - val_accuracy: 0.9967
Epoch 8/50
23